This is a continuation of the 04b. Pandas (With Solutions) Notebook

In [1]:
import pandas as pd
import numpy as np

import os, boto3, subprocess, re, sys, gc
from botocore.client import Config

print("All libraries successfully loaded!")

kms_key = os.environ['AW_S3_ENCRYPTION_KEY']

bucket_name = os.environ['AW_S3_STORAGE_BUCKET']
storage_key = os.environ['AW_S3_STORAGE_KEY'] + '/awdata/rawfiles/'
full_s3_location = 's3://' + bucket_name + '/' + storage_key 
print("full_s3_location: '{}'".format(full_s3_location))
df_twn= pd.read_csv(full_s3_location + "UCI_Credit_Card.csv",nrows=100)
should_be_str = ['SEX','EDUCATION', 'MARRIAGE','PAY_0','PAY_2','PAY_3','PAY_4','PAY_5','PAY_6','default.payment.next.month']
df_twn[should_be_str] = df_twn[should_be_str].astype('str')

To assign a new column to a DataFrame we could use the general syntax: 
````
df['NewColumn'] = Values
df['NewColumn'] = df['Col1'] + df['Col2'] # other calculation
````

Similarly, to delete a column we can use `del`: 
````
del df['NewColumn']
````
If we want to delete several columns at once, we can use `drop`. 
````
df.drop(['Column1',...,'ColumnN'],axis=1,inplace=True)
````
The method `inplace` in the expression above indicates that the DataFrame is <b>permanently</b> modified (in fact, it deletes the original dataframe and constructs a new one with the new desired modification). When we set `axis = 1`, that indicates the dimension of the columns, rather than the rows (`axis = 0`). If in the previous expression there was no `inplace` (e.g. the default of inplace=False), the original dataframe would have kept the dropped columns, as they would simply perform the desired function but not modify the data. 

For example, we can add a new column named <i>Company</i> to an existing dataframe and assign "FICO" as the value, as shown below:


In [3]:
df = pd.DataFrame({'id':[1,2,3,4,5],'name':['Calvin','Ron','Daisy','Jasmine','Juan'],'income':[20000,40000,20000,10000,50000]})
df


In [4]:
df['Company'] = 'FICO'
df

Add 3 new columns to df_twn:

1. **PCT_PAY** =  **PAY_AMT1** divide by **BILL_AMT1**
2. **EDUCATION_MARRIAGE** = Combine the two variables in a single string
3. **Categ_LIMIT**:
    * If  LIMIT_BAL < 100000 then "LOW"
    * Else if LIMIT_BAL < 300000 then "MEDIUM"
    * Else "HIGH"

In [6]:
df_twn['PCT_PAY'] = df_twn['PAY_AMT1']/df_twn['BILL_AMT1']
print(df_twn['PCT_PAY'])


In [7]:
df_twn['EDUCATION_MARRIAGE'] = (df_twn['EDUCATION'].astype("str") + df_twn['MARRIAGE'].astype("str")).astype("str", inplace = True)
df_twn[['EDUCATION','MARRIAGE','EDUCATION_MARRIAGE']]

In [8]:
df_twn['Categ_LIMIT'] = 'HIGH'
df_twn.loc[df_twn['LIMIT_BAL'] < 300000, 'Categ_LIMIT'] = 'MEDIUM'
df_twn.loc[df_twn['LIMIT_BAL'] < 100000, 'Categ_LIMIT'] = 'LOW'
print(df_twn['Categ_LIMIT'].describe())
z.show(df_twn[['LIMIT_BAL','Categ_LIMIT']])

If we want to delete a column (e.g. "Company") from df, one way to do this is with the python (not pandas) keyword `del`

The `del` keyword is used to delete objects. In Python everything is an object, so the del keyword can also be used to delete columns (series), dataframes, etc.


In [10]:
df = pd.DataFrame({'id':[1,2,3,4,5],'name':['Calvin','Ron','Daisy','Jasmine','Juan'],'income':[20000,40000,20000,10000,50000]})
print("original:\n",df)
del df['name']
print("after del: \n",df)


In [11]:
df1 = pd.DataFrame(np.arange(16).reshape(4,4),columns=['col1','col2','col3','col4'],index=['ind1','ind2','ind3','ind4'])
print("original: \n",df1)
df1.drop(['col1','col2'],axis=1,inplace=True)
print("\nafter drop: \n",df1)

Pandas has 3 different types of ``map`` methods:

1. **applymap**:
    * This method works only for **DataFrames** and will apply a function __element-wise__
    * Accepts functions

2. **apply**:
    * This method works for both **Series** and **Dataframe** and will apply a function to a column or row (if it's a DataFrame).
    * Works element-wise and can be used for data aggregation
    * Accepts functions

3. **map**:
    * This method works only for **Series** and is better used to substitute all the values with another ones
    * Accepts dicts, Series or function objects

In [13]:
df_twn[["PAY_AMT1",'BILL_AMT1']].applymap(lambda x: len(str(x)))

In [14]:
df_twn[["PAY_AMT1",'BILL_AMT1']].apply(lambda x: (x - np.mean(x))/np.std(x))

In [15]:
df_twn[["PAY_AMT1",'BILL_AMT1']].apply(lambda x:(np.mean(x))/np.std(x))

In [16]:
df_twn[['PAY_AMT1','PAY_AMT2','PAY_AMT3','PAY_AMT4','PAY_AMT5']].apply(np.sum)

In [17]:
df_twn[['PAY_AMT1','PAY_AMT2','PAY_AMT3','PAY_AMT4','PAY_AMT5']].apply(np.sum, axis = 1)

In [18]:
df_twn['AGE'].apply(lambda x: str(x) + " year")

In [19]:
df_twn['AGE'].map(lambda x: str(x) + " year")

In [20]:
df_twn['SEX']

In [21]:
df_twn['SEX'].map({"1":"Sex_1","2":"Sex_2"})
# df_twn['SEX'].map({"1":"Sex_1"})

Consider the following random pandas DataFrame: 
````
df = pd.DataFrame(np.random.randn(16).reshape(4,4))
````
1. Create an <b>anonymous</b> `lambda` general function, that computes the difference between the mininum (`min()`) and maximum (`max()`) values for any given array (or list of values).
2. Using the result above, apply the function on the previous DataFrame (use `axis = 1`).
3. Repeat the exercise above, but this time apply the function on `axis = 0`. 
4. What is the difference between the results? 


In [23]:
 

df = pd.DataFrame(np.random.randn(16).reshape(4,4))

print('DataFrame is: \n',df)


## defining an anonymous lambda function

f = lambda x: x.max() - x.min()


In [24]:
 
print('axis = 0')
print(df.apply(f))

# df.apply(lambda x: x.max() - x.min())
print(' ')
print('axis = 1')
print(df.apply(f,axis=1))

Setting `axis = 0` calculates the difference between maximum and minimum values for each row whereas `axis = 1` calculates the difference to each column.

<b>Note:</b> there are statistical functions that can be used instead of `apply` (`mean()`, `sum()`, `max()`,`min()`, `std()`, etc.).

Considering the twn_df dataframe, imagine that we want to create a new function `PAY_02` where:

* If 'PAY_0' equal then 'PAY_2' then combine
* else return "0"



In [27]:
df_twn['PAY_02'] = df_twn[['PAY_0','PAY_2']].apply(lambda x: x['PAY_0'] + x['PAY_2'] if x['PAY_0'] == x['PAY_2'] else "0", axis = 1)

Comparing records with `AGE` greater than 35 vs. those with `AGE` less than or equal to 35, which group has a higher average LIMIT_BAL?

In [29]:
print("Age > 35: ",df_twn.loc[df_twn['AGE'] > 35, 'LIMIT_BAL'].mean())
print("Age <= 35: ",df_twn.loc[df_twn['AGE'] <= 35, 'LIMIT_BAL'].mean())

An important operation is "sorting". It involves sorting a specific column using some criterion (i.e. highest to lowest value or vice-versa). Pandas provides some methods for sorting, such as 

```Python
# Sort index or column labels.
sort_index(ascending=True, axis = 0, inplace = False)

# Sort values from columns
sort_values(ascending=True, axis = 0,  inplace = False)

# Compute numerical data ranks (1 to n)
rank(axis = 0)
```
For example, we can sort the following dataframe by its column names: 

````
df = pd.DataFrame(np.arange(8).reshape((2, 4)),index=['three', 'one'],columns=['d', 'a', 'b', 'c'])
````


In [31]:
 

df = pd.DataFrame(np.arange(8).reshape((2, 4)),index=['three', 'one'],columns=['d', 'a', 'b', 'c'])

df


In [32]:
df.sort_index(axis=1)

In [33]:
df = pd.DataFrame({'a':[1,5,4,7,-3],'b':[55,32,-66,-44,1],'c':[1,6,3,33,-55]})
df

In [34]:
print(df.sort_values(by=['a','b']))
print("\nThis is df: \n",df)


In [35]:
df = pd.DataFrame({'a': [5.2, 6, -3, 2], 'b': [1, 1, 1, 1],'c': [-2, 5, 8, -2.5]})
print("df: \n",df)
print("\ndf.rank():\n",df.rank())


* First select the 10 rows with the highest PAY_AMT1. 
* From them, select the 2 with the lowest BILL_AMT1

In [37]:
first_sort = df_twn.sort_values('PAY_AMT1', ascending = False)[0:10]
second_sort = first_sort.sort_values('BILL_AMT1')[0:2]
print(second_sort)

The last - but not the least - topic covered in the `pandas` module is data aggregation and "GroupBy" operations. Data aggregation refers to transformations that produce scalar values (e.g. mean, count, sum, etc. - as we've seen before). The general syntax to group a dataframe by some particular column and apply a function to another particular column is as follows: 
````
df.groupby(['VAR_1',...'VAR_N'])[['var_1',...,'var_n']].agg([function_1,...,function_n])
````
In this section we will look at the following applications of the `groupby` function: 

1. **iterations over groups**
2. **groupby operations using dictionaries and Series**
3. **groupby operations using functions**
4. **groupby operations using index levels**


In [39]:
 
df = pd.DataFrame({'ID' : ['X','X','Y','Z','A','B','Z','X','Y','Z'],
                   'State' : ['Ohio','California','California','Ohio','Ohio','Arkansas','Hawaii','Hawaii','Arkansas','California'],
                   'data1' : np.random.randn(10),
                   'data2' : np.random.randn(10)})
print(df)


Create a GroupBy object to group df by the values in the "ID" column


In [41]:
df.groupby(['ID'])

The object we just created can be used to apply several computations in a dataframe. Calculate the median of the numerical columns by grouping the dataframe by State.


In [44]:
df.groupby(['State']).median()


Group the dataframe by 'State' and calculate the median value of the column 'data2' (only)



In [47]:
 
print(type(df['data2']))
print(type())
df['data2'].groupby(df['State']).median()


What's the difference between both syntaxes (grouping the entire dataframe versus a single column) shown above in questions 15 & 16? 


In Answer 15:  
* We use pandas.<font color=red>DataFrame</font>.groupby
* We get the desired groups (by State) by specifying them as a list.

In Answer 16:
* We use pandas.<font color=red>Series</font>.groupby
* We get the desired groups (by State) by specifying them as a Series.

Compute the median of column 'data2' by both keys (the combination of ID and State).


In [53]:
 

df['data2'].groupby([df['ID'],df['State']]).median()


 

Count the number of records within each unique combination of ID and State.


In [56]:
 

df.groupby(['ID','State']).count()


The following syntax allows one to visualize and apply complex operations to each pair name-group in a GroupBy object: 

````
for name, group in df.groupby('ID'): 
...
````

What would be the code to loop over a `groupby` object in the case of multiple keys? (Assume the keys are 'ID' and 'State').


In [59]:
 

for (i,j), group in df.groupby(['ID','State']): 
    print(i,j)
    print(group)

We have seen the usage of dictionaries applied to dataframes. Convert the groupby object we just created into a dictionary to visualize the 'pieces' of the previous script in dictionary format.

In [62]:
 
print(df,"\n")
d =dict(list(df.groupby(['ID','State'])))

print("d.keys():\n",d.keys(),"\n")

print("type(d.values()):\n",type(d.values()),"\n")

print("d.items()\n",d.items(),"\n")

We can can also select a subset of columns when using groupby such as : 

````
df.groupby('State')['ID']
````
Compute the `mean` of data1 using the previous syntax applied to State and ID.




In [65]:
 

df.groupby(['State','ID'])['data1'].mean()

Another form of applying `groupby` using a dictionary is by mapping the values in a dictionary such as: 
````
mapping = {'Col1':'Column1','Col2':'Column2','Col3':'Column3'}
df.groupby(mapping,axis=1)
````
Use the dictionary below to compute the sum: 
````
my_dict = {'data1':'BirthRate','data2':'DeathRate','data3':'AssaultRate'} 
````


In [68]:
 
print(df,"\n")
my_dict = {'data1':'BirthRate','data2':'DeathRate','data3':'AssaultRate'} 

df.groupby(my_dict,axis=1).sum()


It is possible to calculate several different functions using `groupby`. The method implemented in pandas is: 
````
df.groupby('key').agg('function')
````
We can also apply a list of functions to be computed, such as: 

````
[function1, function2, ..., functionN]
````

a) Write a function to compute the distance between the maximum value and the mean value of the data columns for each US State.

b) Compute the mean, median, maximum, minimum (for each numeric column) and count the values for each 'ID'.


In [71]:
def MaxToMean(data):
    return abs(round(data.max() - data.mean(),2))

print(df.groupby('State').agg(MaxToMean))



In [72]:
print(df.groupby('ID').agg(['mean','median','max','min','count']))

Compute (on a single column!) all the statistics provided by the previous functions only for ID=X. 


In [74]:
 
print(df)
print(df.groupby('ID').agg(['mean','median','max','min','count']).transpose()['X'],"\n")
print(df.groupby('ID').agg(['mean','median','max','min','count']).loc["X",:])


You can also use tuples to convert the name of the column computed into something else. Change the previous code to print 'Function1', ... , 'FunctionN' to each of the functions computed before grouped by ID for all IDs.

In [77]:
FuncsList = [('Function1','mean'),
             ('Function2','median'),
             ('Function3','max'),
             ('Function4','min'),
             ('Function5','count')]

df.groupby('ID').agg(FuncsList).transpose()

It is also possible to apply different functions to different columns using a dictionary. 

For each value of State:

* Calculate the maximum, minimum and count of the data1 column
* Calcykate the mean, median and the function we defined (MaxToMedian) of the data2 column.


In [80]:
 

funcs_dict = {'data1':['min','max','count'],'data2':['mean','median',MaxToMean]}

df.groupby('State').agg(funcs_dict).transpose()


We have previously seen the 'apply' method, useful to compute functions in a given column. Its capabilities are stronger when used together with GroupBy. 

a) Write a function such as : $$f(x,n) = [x_1, x_2, ..., x_n]^2$$ to compute the square of each element in a list, where n is any length we want, restricted by the maximum number of rows in the dataframe.

b) Use the function defined above and `apply` to both data1 and data2 of each US State. (do it iteratively starting with \\(n = 1\\) until n = number of rows).



In [83]:
def Squared(x,n=1): 
    return (x**2)[-n:]

In [84]:
nRows = df.shape[0]
for i in range(nRows): 
    print(i + 1)
    df.groupby('State')['data1','data2'].apply(Squared, n = (i+1))
    


 

Answer the follow questions using df_twn

1. Which 'EDUCATION' category has the highest average PAY_AMT1?
2. Which 'MARRIAGE' category has the highest number of records with "PAY_0" equal 0 ?
3. Create a table containing at least 6 statistical functions to analyze the "LIMIT_BALL" by "EDUCATION"

In [87]:
df_twn.groupby("EDUCATION")['PAY_AMT1'].mean()

In [88]:
df_gb = df_twn.groupby(["MARRIAGE","PAY_0"])[["ID"]].count().reset_index()
df_gb = df_gb.loc[df_gb['PAY_0'] == '0', :]
df_gb

In [89]:
z.show(df_twn.groupby('EDUCATION')['LIMIT_BAL'].agg(['mean','median','max','min','count','std','var']).reset_index())